In [ ]:
import rp
from easydict import EasyDict
from source.peekaboo import make_image_square
from rp import ic
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
outputs_folder='untracked/dep_peekaboo_RefMatte'

In [ ]:
result_folders=outputs_folder
result_folders=rp.get_all_folders(result_folders)
result_folders=[rp.get_all_folders(x) for x in result_folders]
result_folders=rp.list_flatten(result_folders)

In [ ]:
result_folders[0]

In [ ]:
rp.ic(len(result_folders))

In [ ]:
class Result:
    def __init__(self, path):
        self.path=path
        
        self.trial_num=int(rp.get_folder_name(self.path)) # Like 000, 001 etc
        self.params=EasyDict(rp.load_json(rp.path_join(self.path,'params.json')))
        self.prompt=self.params.extra_data.prompt
        self.name=self.params.extra_data.entry.image_name
        self.preset_name=rp.get_folder_name(rp.get_parent_folder(self.path)).split('.')[-1]
        self.square_image_method=self.params.extra_data.square_image_method
        
        self.alpha_path         =rp.path_join(self.path,'alphas','0.png')
        self.image_path         =rp.path_join(self.path,'image.png')
        self.preview_image_path =rp.path_join(self.path,'preview_image.png')
        self.mask_path          =rp.path_join('datasets/RefMatte_RW_100',self.params.extra_data.entry.mask_path)
        self.original_image_path=rp.path_join('datasets/RefMatte_RW_100',self.params.extra_data.entry.image_path)
        self.mask_name=rp.get_file_name(self.mask_path, include_file_extension=False)
        
        self.swappable = '_1.png' in self.mask_path or '_2.png' in self.mask_path
        self.other_mask_path    =rp.path_join('datasets/RefMatte_RW_100',
                                      self.params.extra_data.entry.mask_path
                                      .replace('_1.png','_A.png')
                                      .replace('_2.png','_B.png')
                                      .replace('_A.png','_2.png')
                                      .replace('_B.png','_1.png')
                                     )
        
        assert self.swappable, 'I want to discard these for sanitys sake'

        self.original_mask       =rp.load_image(self.mask_path          ,use_cache=True)
        self.original_other_mask =rp.load_image(self.other_mask_path    ,use_cache=True)
        
        # self.original_mask, self.original_other_mask = self.original_other_mask, self.original_mask #Do the Ol' switcheroo! This is for testing the reverse...
        
        self.original_image      =rp.load_image(self.original_image_path,use_cache=True)
        self.alpha               =rp.load_image(self.alpha_path         ,use_cache=True)
        self.image               =rp.load_image(self.image_path         ,use_cache=True)
        self.preview_image       =rp.load_image(self.preview_image_path ,use_cache=True)
        self.mask      =rp.cv_resize_image(make_image_square(self.original_mask      , self.square_image_method),rp.get_image_dimensions(self.alpha))
        self.other_mask=rp.cv_resize_image(make_image_square(self.original_other_mask, self.square_image_method),rp.get_image_dimensions(self.alpha))
        
        print(self)
        
    def __repr__(self):
        return 'Result(%s, %s, %s)'%(self.mask_name, self.trial_num, self.preset_name)
    
    @property 
    def best_iou(self):
        return get_best_iou(self)
    
    @property 
    def best_other_iou(self):
        return get_best_iou(self,self.other_mask)
    
def try_load_result(path):
    try:
        return Result(path)
    except Exception as e:
        #AttributeError: 'EasyDict' object has no attribute 'extra_data'
        rp.fansi_print(e,'red')

In [ ]:
from rp import *
def get_mask_iou(*masks):
    """Calculates the IOU (intersection over union) of multiple binary masks"""
    masks=detuple(masks)
    assert all(is_image(mask) for mask in masks), 'All masks must be images as defined by rp.is_image'
    assert len(set(get_image_dimensions(mask) for mask in masks))==1, 'All masks must have the same dimensions, but got shapes '+repr(set(get_image_dimensions(mask) for mask in masks))
    masks = as_numpy_array([as_binary_image(as_grayscale_image(mask)) for mask in masks])
    intersection = np.min(masks, axis=0)
    union = np.max(masks, axis=0)
    return np.sum(intersection) / np.sum(union)

def get_iou(result, threshold=.1, mask=None):
    return get_mask_iou(mask if mask is not None else result.mask, rp.as_float_image(result.alpha)>threshold)

@memoized
def get_best_iou(result, mask=None):
    return max(get_iou(result, threshold, mask) for threshold in [.1,.2,.3,.4,.5,.6,.7,.8,.9])

In [ ]:
from tqdm import tqdm
import random
from collections import defaultdict
from typing import List

@memoized
def alpha_filter_1(alpha):
    pred_img=alpha
    alpha=pred_img
    std=alpha.std()
    alpha=alpha-alpha.mean()*.56
    alpha=alpha/std/(2+.1)
    pred_img=alpha
    pred_img=rp.cv_gauss_blur(pred_img,10)
    R=45
    pred_img=rp.cv_dilate(pred_img,R,circular=True)
    # pred_img=rp.cv_erode(pred_img,R-6,circular=True)
    pred_img=rp.cv_erode(pred_img,R,circular=True)
    R=9
    pred_img=rp.cv_gauss_blur(pred_img,80)
    pred_img=rp.cv_erode(pred_img,R,circular=True)
    pred_img=rp.cv_dilate(pred_img,R,circular=True)
    pred_img=rp.as_float_image(pred_img)
    return pred_img

@memoized
def alpha_filter_2(alpha):
    pred_img=alpha
    alpha=pred_img
    std=alpha.std()
    #alpha=alpha-alpha.mean()*.56
    #alpha=alpha/std/(2+.1)
    #pred_img=alpha
    #pred_img=rp.cv_gauss_blur(pred_img,10)
    
    R=40
    pred_img=rp.cv_dilate(pred_img,R,circular=True)
    pred_img=rp.cv_erode(pred_img,R,circular=True)
    # R=10
    # pred_img=rp.cv_gauss_blur(pred_img,10)
    # pred_img=rp.cv_erode(pred_img,R,circular=True)
    # pred_img=rp.cv_dilate(pred_img,R,circular=True)
    pred_img=rp.as_float_image(pred_img)
    return pred_img


def chunkoozle(alpha,r=10,iter=1):
    #Bad name maybe
    alpha=as_grayscale_image(alpha)
    alpha=as_float_image(alpha)
    a=alpha
    for _ in range(iter):
        oa=a
        a=cv_dilate(a,r,circular=True)
        a=cv_erode(a,r,circular=True)
        a=as_float_image(a)
        a=np.maximum(a,oa)
    return a

@memoized
def alpha_filter_3(alpha):
    alpha=chunkoozle(alpha,60,2)
    alpha=rp.as_float_image(alpha)
    return alpha

@memoized
def alpha_filter_4(alpha):
    alpha=chunkoozle(alpha,90,3)
    alpha=rp.as_float_image(alpha)
    return alpha

@memoized
def alpha_filter_5(alpha):
    alpha=chunkoozle(alpha,70,1)
    alpha=rp.as_float_image(alpha)
    return alpha


In [ ]:
results = [x for x in rp.par_map(try_load_result, result_folders) if x is not None]

In [ ]:
best_results = sorted(results, key=get_best_iou, reverse=True)
best_results = [x for x in best_results if x.square_image_method=='scale']

In [ ]:
for i in range(10):
    bri=best_results[i]
    print(get_best_iou(bri))
    rp.display_image(rp.horizontally_concatenated_images(bri.mask,bri.preview_image))
    print()
    print()

In [ ]:
for x in best_results[:10]:
    print(x.mask_path)

In [ ]:
#REVERSED
for i in range(20):
    bri=best_results[-i-1]
    print(get_best_iou(bri))
    rp.display_image(rp.horizontally_concatenated_images(bri.mask,bri.preview_image))
    print()
    print()

In [ ]:
plot_points = [
    (x.best_iou, x.best_other_iou) for x in results 
    if (
        True
        # x.preset_name=='midas_raster_bilateral_low_grav_bilat0rgb_sd15__200iter' 
        # and 
        # x.square_image_method=='scale'
    )
]

x,y=zip(*plot_points)
x=as_numpy_array(x)
y=as_numpy_array(y)

rp.scatter_plot(plot_points, xlabel='IOU', ylabel='Other IOU')


#############
import seaborn
sns=seaborn
# seaborn.regplot(x=x,y=y)
import matplotlib.pyplot as plt
seaborn.regplot(x=x, y=y, scatter_kws=dict(color='blue', s=10, alpha=.5, edgecolor='none'), line_kws=dict(color='green',linewidth=1.5, alpha=0.5, linestyle='--'), )
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel('IOU')
plt.ylabel('Other IOU')
plt.show()


###########

# Create the jointplot with regression
g = seaborn.jointplot(x=x, y=y, kind='reg',
                  joint_kws={'scatter_kws': {'color': 'black', 's': 10, 'edgecolor': 'none'},
                             'line_kws': {'color': 'blue', 'linewidth': 1, 'alpha': 0.5, 'linestyle': '--'}},
                  marginal_kws={'bins': 10, 'color': 'black'}
                 )

# Set axis labels
g.ax_joint.set_xlabel('IOU')
g.ax_joint.set_ylabel('Other IOU')
# Set axis limits for the scatter plot
g.ax_joint.set_xlim(0, 1)
g.ax_joint.set_ylim(0, 1)
plt.show()


#####################

# Create the jointplot with regression
g = sns.jointplot(x=x, y=y, kind='reg',
                  joint_kws=dict(scatter_kws=dict(color='blue', s=10, alpha=0.5, edgecolor='none'),
                                 line_kws=dict(color='green', linewidth=1.5, alpha=0.5, linestyle='--')),
                  marginal_kws=dict(bins=10, color='black'),
                                    # Adjust the size of the histograms relative to the scatter plot
                  height=6, ratio=4
                 )

# Adjust the size of the histograms to make them taller
g.fig.subplots_adjust(top=0.9, bottom=0.1, left=0.1, right=0.9)

# Set axis labels
g.ax_joint.set_xlabel('IOU')
g.ax_joint.set_ylabel('Other IOU')

# Set axis limits for the scatter plot
g.ax_joint.set_xlim(0, 1)
g.ax_joint.set_ylim(0, 1)

plt.show()



####################
g = sns.jointplot(x=x, y=y, kind='reg',
                  joint_kws=dict(scatter_kws=dict(color='blue', s=10, alpha=0.5, edgecolor='none'),
                                 line_kws=dict(color='green', linewidth=1.5, alpha=0.5, linestyle='--')),
                  marginal_kws=dict(bins=10, color='black'),
                  height=6, ratio=4
                 )

# Set axis labels
g.ax_joint.set_xlabel('IOU')
g.ax_joint.set_ylabel('Other IOU')

# Set axis limits for the scatter plot
g.ax_joint.set_xlim(0, 1)
g.ax_joint.set_ylim(0, 1)

# Plot the mean for x and display it on the marginal x plot
mean_x = sum(x) / len(x)
g.ax_marg_x.axvline(mean_x, color='red', linestyle='--')
g.ax_marg_x.text(mean_x + 0.02, 0.5, f"Mean={mean_x:.4f}", verticalalignment='center', color='red', transform=g.ax_marg_x.transAxes)

# Plot the mean for y and display it on the marginal y plot
mean_y = sum(y) / len(y)
g.ax_marg_y.axhline(mean_y, color='red', linestyle='--')
g.ax_marg_y.text(0.5, mean_y + 0.02, f"Mean={mean_y:.4f}", horizontalalignment='center', color='red', transform=g.ax_marg_y.transAxes)

plt.show()



###########
# Create the jointplot with regression
g = sns.jointplot(x=x, y=y, kind='reg',
                  joint_kws=dict(scatter_kws=dict(color='blue', s=10, alpha=0.5, edgecolor='none'),
                                 line_kws=dict(color='green', linewidth=1.5, alpha=0.5, linestyle='--')),
                  marginal_kws=dict(bins=10, color='#add8e6'),  # Light blue for bins
                  height=6, ratio=4
                 )

# Set axis labels
g.ax_joint.set_xlabel('IOU')
g.ax_joint.set_ylabel('Other IOU')

# Set axis limits for the scatter plot
g.ax_joint.set_xlim(0, 1)
g.ax_joint.set_ylim(0, 1)

# Plot the mean for x and display it on the marginal x plot
mean_x = sum(x) / len(x)
qw=-9
g.ax_marg_x.axvline(mean_x, color='black', linestyle='--')
g.ax_marg_x.text(mean_x + 0.02, g.ax_marg_x.get_ylim()[1] - qw, f"μ={mean_x:.3f}".lstrip('0'), color='black', verticalalignment='top')

# Plot the mean for y and display it on the marginal y plot
mean_y = sum(y) / len(y)
g.ax_marg_y.axhline(mean_y, color='black', linestyle='--')
g.ax_marg_y.text(g.ax_marg_y.get_xlim()[1] - qw, mean_y + 0.02, f"μ={mean_y:.3f}".lstrip('0'), color='black', horizontalalignment='right', rotation=-90)

plt.show()




In [ ]:
ious=[get_best_iou(result) for result in results]
sorted(ious)
import matplotlib.pyplot as plt

def display_histogram(scores: list, title='Histogram of IOUs', cum=False):
    # Create a histogram with 10 bins
    plt.hist(scores, bins=10, range=(0, 1), edgecolor='black', density=True, cumulative=cum)

    # Add labels and title
    plt.xlabel('IOU')
    plt.ylabel('Probability Density' if not cum else 'Cumulative Probability Density')
    plt.title(title)

    # Display the histogram
    plt.show()
    
display_histogram(ious)
display_histogram(ious,cum=True)
ic(rp.median(ious),rp.mean(ious))

In [ ]:
clusters=cluster_by_key(results, key=lambda x:(x.preset_name, x.square_image_method),as_dict=True)
rp.pretty_print({cat:mean(x.best_iou for x in clusters[cat]) for cat in clusters})

In [ ]:
cluster_ious=[x.best_iou for x in clusters[('midas_raster_bilateral_low_grav_bilat0rgb_sd15__200iter', 'scale')]]
display_histogram(cluster_ious,'Peekaboo: Mask IOU')
display_histogram(cluster_ious,'Peekaboo: Mask IOU', cum=True)
ic(rp.median(cluster_ious),rp.mean(cluster_ious))

cluster_ious=[x.best_other_iou for x in clusters[('midas_raster_bilateral_low_grav_bilat0rgb_sd15__200iter', 'scale')]]
display_histogram(cluster_ious,'Peekaboo: Opposite Mask IOU')
display_histogram(cluster_ious,'Peekaboo: Opposite Mask IOU',cum=True)
ic(rp.median(cluster_ious),rp.mean(cluster_ious))

In [ ]:
len(best_results)

In [ ]:
groupvit_scores=[0.0, 0.31580823304857897, 0.24013550287034965, 0.30591783989862253, 0.6459746085440096, 0.011801912842415035, 0.004670268010047687, 0.25356588147666276, 0.48194745108026654, 0.4180502684657809, 0.018789298562887354, 0.0, 0.0, 0.0017302535562752329, 0.3539054948794341, 0.25356588147666276, 0.36439199257357247, 0.0017302535562752329, 0.1710613438827937, 0.0, 0.0, 0.017779906999907974, 0.323933824425187, 0.5101056434334685, 0.0, 0.12872303200906637, 0.0926172879511115, 0.26369738664370246, 0.3547532757178701, 0.038441451828467595, 0.3383857058556308, 0.01033767367735455, 0.2746005333819607, 0.4434515290229311, 0.2746005333819607, 0.0001678331947835345, 0.6947673973022686, 0.0, 0.8197329004401452, 0.06572361762811667, 0.0, 0.4800289662028659, 0.21103973107025378, 0.38483763138042637, 0.2807044395291887, 0.0, 0.5916879598559246, 0.347458213584961, 0.4653205550956263, 0.14394945153421065, 0.1859965237758169, 0.5155389977492297, 0.2805174206817218, 0.5155389977492297, 0.0, 0.008583532425309721, 0.0001454389093346486, 0.5013221952656007, 0.07570677357213516, 0.4054745161852734, 0.526253588826395, 0.03522840727805038, 0.09069696279742266, 0.15118094179167163, 0.5242999644393089, 0.2287069005010141, 0.5200174076945469, 0.3959092267039678, 0.5013221952656007, 0.0015118765483310924, 0.26369738664370246, 0.03867130302707173, 0.05304178584599531, 0.027385156443010567, 0.0, 0.0, 0.0, 0.13220653684011419, 2.2718495126882797e-05, 0.2868756705703006, 0.03259445455569454, 0.18294698289338515, 0.2779267001554299, 0.00546333866142055, 0.1707269556774849, 0.44152368552472, 0.4949829406549714, 0.001001575906146034, 0.01698056245790855, 0.27119934261653905, 0.0, 0.0005086220631671834, 0.3965067012229576, 0.4247535702522273, 0.5451280734782361, 0.2791880535215935, 0.015707108621585725, 0.34704848119623594, 0.1859965237758169, 0.16759288400059633, 0.0028169014084507044, 0.0, 0.3775903608996312, 0.0, 0.16759288400059633, 0.1804846826426772, 0.011801912842415035, 0.1173135437450181, 0.0, 0.7282061380768157, 0.3591660879356496, 0.0, 0.0006819959463865937, 0.36245548721269016, 0.3480875686265576, 0.24790527876282464, 0.0, 0.16328325909678706, 0.7282061380768157, 0.0015488784405916349, 0.40001924278602435, 0.0, 0.0029414042814835924, 0.34870356365434885, 0.05082332573460867, 0.003636408787642998, 0.7028389182352384, 0.4560551692516166, 0.644969070254056, 0.13220653684011419, 0.0, 0.4180502684657809, 0.05185448441414412, 0.28442006448525226, 0.38483763138042637, 0.21639652597222991, 0.03843237587736205, 0.2951603775608356, 0.45713271158484375, 0.4572000959069125, 0.44531505830143847, 0.01067737641608062, 0.36369940183440214, 0.0, 0.0023673503678544425, 0.0, 0.05185448441414412, 0.11947958882287718, 0.8197329004401452, 0.3742341028274319, 0.011801912842415035, 0.0007329887751062031, 0.01913418959999733, 0.002740537099284471, 0.3538410785069509, 0.16021802784917036, 0.3804733443055275, 0.4516648028104692, 0.09069696279742266, 0.3977559548901136, 0.1108124498699063, 0.07570677357213516, 0.3531309320953426, 0.35682150698964005, 0.007737097095999403, 0.5928567476345931, 0.017372631960209817, 0.00225105460456782, 0.19932701150853818, 0.3539054948794341, 0.24783070282742092, 0.2791880535215935, 0.38780010350336386, 0.05207760177627022, 0.0002476795996272039, 0.1405386141665543, 0.04972816644060577, 0.0, 0.0, 0.3531309320953426, 0.0, 0.3044844124933116, 0.8489511152174171, 0.6506216558168465, 0.04076075013713882, 0.5431607600900382, 0.47910314669657217, 0.026355546734820347, 0.0, 0.1084803221597855, 0.08190884860990613, 0.27017365176686925, 0.3531309320953426, 0.45870082103235704, 0.09069696279742266, 0.2746005333819607, 0.08006163605833679, 0.03843237587736205, 0.5431607600900382, 0.0, 0.5451280734782361, 0.008583532425309721, 0.004670268010047687, 0.0, 0.0, 0.05304178584599531, 0.001362072805563117, 0.017372631960209817, 0.13220653684011419, 0.5002194048281937, 0.22961024820677092, 0.45870082103235704, 0.762555998816901, 4.937101329067678e-05, 0.0, 0.578767340428017, 0.05207760177627022, 0.0, 0.0006973251627233779, 0.39911056913149023, 0.0, 0.12468878036848405, 0.3538410785069509, 0.004813814974973129, 0.9287301900078131, 0.0, 0.35052069575851863, 0.026355546734820347, 0.12783237330801137, 0.003057938283563648, 0.3044844124933116, 0.2746005333819607, 0.03693211025942723, 0.3733530304240198, 0.3775903608996312, 0.2807044395291887, 0.8885500232049327, 0.5065104925981155, 0.0, 0.0002476795996272039, 0.6947673973022686, 0.24783070282742092, 0.0, 0.3499426878193325, 0.01913418959999733, 0.3338747226302998, 0.4138532153662632, 0.11947958882287718, 0.0, 0.12783237330801137, 0.006442907469054006, 0.5208633475320329, 7.157560111164313e-05, 0.0, 0.2868756705703006, 0.027385156443010567, 0.017953303452731034, 0.0, 0.00663011448722284, 0.004813814974973129, 0.3012417272786842, 0.0017302535562752329, 0.00546333866142055, 0.0021873941767984553, 0.526253588826395, 0.0, 0.008583532425309721, 0.017953303452731034, 0.20075899248100906, 0.0926172879511115, 0.11315818680534635, 0.028820090866121983, 0.43282963485534676, 0.0, 0.038441451828467595, 0.04076075013713882, 0.4828225591868223, 0.25097942958785857, 0.3547532757178701, 0.0022721559375785587, 0.0, 0.4544662193405877, 0.45218841467565174, 0.11947958882287718, 0.40368495828890893, 0.0029414042814835924, 0.3993516070174706, 0.0001678331947835345, 0.014204736255322578, 0.36071069014866614, 0.3338747226302998, 0.3749654434655096, 0.3149549514498352, 0.40538629173989454, 0.3464607483820884, 0.501873096321369, 0.003904913347154848, 0.4516648028104692, 0.862217832606562, 0.4949829406549714, 0.3468345980242649, 0.1109451882943695, 0.23447798135147335, 0.004473138259631626, 0.038441451828467595, 0.20075899248100906, 0.4784169421599272, 0.04076075013713882, 0.003057938283563648, 0.36992635902366905, 0.29309310476320016, 0.05185448441414412, 0.0015821730248192526, 0.26731734313028815, 0.16759288400059633, 0.03259445455569454, 0.003904913347154848, 0.0, 0.22961024820677092, 0.4784169421599272, 0.002652449642521827, 0.862217832606562, 0.4800289662028659, 0.5065104925981155, 0.0, 0.8489511152174171, 0.4949829406549714, 0.15119852787663685, 0.347458213584961, 0.00225105460456782, 0.26253798308301063, 0.6345074401522766, 0.5174267676894638, 0.0, 0.4480862529132353, 0.331686309451487, 0.019221138605938264, 0.017396385019884275, 0.12872303200906637, 0.26992809184894906, 0.0, 0.09069696279742266, 0.4180502684657809, 0.6119127802156683, 0.09834626128367928, 0.04972816644060577, 0.40538629173989454, 0.3069206917731683, 0.005041885734137253, 0.1733234315973007, 0.0, 0.20170983424074873, 0.05082332573460867, 0.39370901552650506, 0.10795830884708592, 0.4434515290229311, 0.4560551692516166, 0.2076885159766647, 0.3550526379113277, 0.41520969086225185, 0.4480862529132353, 0.38132999266579337, 0.017396385019884275, 0.11947958882287718, 0.026355546734820347, 0.39620717587077714, 0.027385156443010567, 0.11085632594710508, 0.3759277416118477, 0.0, 0.0, 0.0031009053027241966, 0.01106189326692965, 0.0005669537555365065, 0.16759288400059633, 0.0022721559375785587, 0.12872303200906637, 0.0006586404039200757, 0.0005669537555365065, 0.3733530304240198, 0.8200153990765412, 0.0, 0.0, 0.00034479221904344166, 0.044348037900721816, 0.4247535702522273, 0.3550526379113277, 8.839654690943573e-05, 0.03986370570410635, 0.0031009053027241966, 0.11315818680534635, 0.0, 0.4762734283761073, 0.5155389977492297, 0.32251910860920313, 0.022378834319101554, 0.2919168156160224, 0.18067037956826498, 0.3838459592877303, 0.31849292128500445, 0.0026172273928197834, 0.31849292128500445, 0.1710613438827937, 0.36439199257357247, 0.0, 0.08659923961928363, 0.08031143475771152, 0.0031009053027241966, 0.04985272334122214, 5.219733961946456e-05, 0.15033858031130232, 0.26253798308301063, 0.0005669537555365065, 0.0014931753278751788, 0.0013518968263023512, 0.0, 0.4653205550956263, 0.0, 0.1859965237758169, 0.3471421800814958, 0.18710987507173443, 0.27251382562553955, 0.04985272334122214, 0.013212233697949627, 0.5101056434334685, 0.669067131740209, 0.0038150142565597086, 0.8904937617222539, 0.015084109870747063, 0.0, 0.48244518635035794, 0.0, 0.25356588147666276, 0.16021802784917036, 0.014204736255322578, 0.49370337723426144, 0.007248339823541584, 0.7282061380768157, 0.0, 0.21141461479850115, 0.017396385019884275, 0.08190884860990613, 0.0, 0.36071069014866614, 0.42159645815458546, 0.36439199257357247, 0.4480862529132353, 0.5388925670278284, 0.3044844124933116, 0.006442907469054006, 0.0, 0.3464607483820884, 0.2807044395291887, 0.16054330005273712, 0.0, 0.024970785596395484, 0.1710613438827937, 0.0005102581856516397, 0.014204736255322578, 0.01033767367735455, 0.9238301398589572, 0.2919168156160224, 0.03986245176851262, 0.30963048638536994, 0.38543578784004684, 0.3687372820893879, 0.0, 0.0, 0.0, 0.09426367464257568, 0.0, 0.24783070282742092, 0.3468345980242649, 0.049901517312577255, 0.05912480662801578, 0.021773548862657158, 0.19932701150853818, 0.5030869408059292, 0.40911047309443155, 0.2791880535215935, 0.3468345980242649, 0.4180502684657809, 0.015244881148666277, 0.017779906999907974, 0.875635801002662, 0.0013518968263023512, 0.5190675676927209, 0.06928973513149098, 0.0, 0.0, 0.7213176708804582, 0.0, 0.0, 0.0, 0.1804846826426772, 0.3968527566280598, 0.1914474421755402, 0.0, 0.2287069005010141, 0.28442006448525226, 0.31849292128500445, 0.40001924278602435, 0.40001924278602435, 0.3733530304240198, 0.024970785596395484, 0.0813086418829724, 0.48555638860523737, 0.4773677164132388, 0.0, 0.862217832606562, 0.0006686600305312693, 0.43024869077279276, 0.4055575226252355, 0.002652449642521827, 0.35441230757195347, 0.05207760177627022, 0.4260063654743079, 0.3993516070174706, 0.03843237587736205, 0.3291414398283298, 0.7282061380768157, 0.00225105460456782, 0.28442006448525226, 0.42159645815458546, 0.45341215309971605, 0.4773677164132388, 0.35676710575574144, 0.44531505830143847, 0.0, 0.5549371172595003, 0.6212996921007555, 0.6150448819996139, 0.0926172879511115, 0.00225105460456782, 0.0, 0.6745234679237548, 0.021773548862657158, 0.0, 0.25097942958785857, 0.4487502131010464, 0.6212996921007555, 0.3490713169565571, 0.04076075013713882, 0.03986245176851262, 3.5003917746178515e-05, 0.04182251130312276, 5.219733961946456e-05, 0.6490319120231131, 0.7956569593666694, 0.019221138605938264, 0.3959092267039678, 0.184357324204992, 0.20170983424074873, 0.0, 0.35448309530095695, 0.48217345971038283, 0.0, 0.3687372820893879, 0.0, 0.0005632005683418183, 0.35682150698964005, 0.11085632594710508, 0.0, 0.2379868124738776, 0.0, 0.0, 0.0, 0.04985272334122214, 0.7956569593666694, 0.4233424090922028, 0.005041885734137253, 0.04739252275854451, 0.2247518692014286, 0.04747838101817662, 0.021773548862657158, 0.01033767367735455, 0.3804733443055275, 0.003904913347154848, 0.4480862529132353, 0.0, 0.04985272334122214, 0.5030869408059292, 0.3748898501291643, 0.08031143475771152, 0.4507927199375682, 0.04822693499122674, 0.8200153990765412, 0.4784169421599272, 0.0005669537555365065, 0.0038150142565597086, 0.3789975010499165, 0.03843237587736205, 0.4107524690736708, 0.4475639771276902, 0.002740537099284471, 0.39620717587077714, 0.44790165698409545, 0.30963048638536994, 0.1707269556774849, 0.0, 0.01067737641608062, 0.4949829406549714, 0.0813086418829724, 0.0, 0.001359781775627766, 0.2421624966637615, 0.0006586404039200757, 0.24783070282742092, 0.3687372820893879, 0.6745234679237548, 0.31580823304857897, 0.1733234315973007, 0.347458213584961, 0.3236322008952096, 0.331686309451487, 0.07488649361728125, 0.0, 0.12990606137638705, 0.3539054948794341, 0.015084109870747063, 0.6459746085440096, 4.937101329067678e-05, 0.05207760177627022, 0.5013221952656007, 0.3222567729814179, 0.25097942958785857, 0.04964803963978135, 0.762555998816901, 0.8644689236988378, 0.3044844124933116, 0.0, 0.501873096321369, 0.00022505572929767338, 0.3959092267039678, 0.03522840727805038, 0.04589722187724735, 0.2805216007064674, 0.2720534858085158, 0.5101056434334685, 0.0, 0.0, 0.15033858031130232, 0.36245548721269016, 0.08985497230751005, 0.0, 0.3742341028274319, 0.3627395415487322, 0.23447798135147335, 0.0, 0.8885500232049327, 0.004813814974973129, 0.049901517312577255, 0.0029414042814835924, 0.26992809184894906, 0.04747838101817662, 0.4593313339517174, 0.09426367464257568, 0.3804733443055275, 0.0926172879511115, 0.3489482798557263, 0.4532780145025043, 0.01352479663483049, 0.0, 0.13220653684011419, 0.0, 0.0, 0.3383857058556308, 0.05185448441414412, 0.669067131740209, 0.42861561922541536, 0.2868756705703006, 0.0, 0.3690727786468491, 0.2852911941444901, 0.388240626067142, 0.3116379474684682, 0.38543578784004684, 0.0, 0.0006032388134684738, 0.27017365176686925, 0.21103973107025378, 0.17574493283355527, 0.16328325909678706, 0.0001454389093346486, 0.08938764099401394, 0.024970785596395484, 0.09366047012244036, 0.007737097095999403, 0.3373939902070483, 0.2805174206817218, 0.001362072805563117, 0.0, 0.25097942958785857, 0.007099891268877665, 0.21141461479850115, 0.004824216162293178, 0.8800086772753355, 0.4572000959069125, 0.875635801002662, 0.15390786124884812, 0.01106189326692965, 0.000315645132079377, 0.36245548721269016, 0.2919168156160224, 0.20170983424074873, 0.0014931753278751788, 0.8644689236988378, 0.007645503234819579, 0.36829446138694016, 0.32930956427029545, 0.0, 0.01106189326692965, 0.5190675676927209, 0.5065104925981155, 0.08985497230751005, 0.3804733443055275, 0.762555998816901, 0.011801912842415035, 0.4233424090922028, 0.669067131740209, 0.45870082103235704, 0.47371654995864015, 0.578767340428017, 0.0, 0.0, 0.0005368410318472304, 0.3383857058556308, 0.5065104925981155, 0.4532780145025043, 0.7184412510108106, 0.001242726247189917, 0.7089055979234758, 0.052055956618649986, 0.3748898501291643, 0.264146543336602, 0.4230601383511605, 0.4593313339517174, 0.06324504135316515, 0.5140734818960061, 0.9236403477625719, 0.0, 0.0, 0.1341391188323712, 0.26689970913573974, 0.03522840727805038, 0.0, 0.5191151225937403, 0.004824216162293178, 0.002740537099284471, 0.6743893547211083, 0.5242999644393089, 0.25356588147666276, 0.526253588826395, 0.19216629143295896, 0.6212996921007555, 0.10795830884708592, 0.4773677164132388, 0.004824216162293178, 0.4516648028104692, 0.03522840727805038, 0.11643918158196231, 0.16021802784917036, 0.0, 0.004824216162293178, 0.0015488784405916349, 0.0031009053027241966, 0.4260063654743079, 0.0014931753278751788, 0.17346167678947388, 0.04747838101817662, 0.04182251130312276, 0.049901517312577255, 0.08985497230751005, 0.002652449642521827, 0.00663011448722284, 0.0, 0.04589722187724735, 0.0, 0.5140734818960061, 0.005041885734137253, 0.0, 0.044348037900721816, 0.003904913347154848, 0.20075899248100906, 0.011648795432391033, 0.35441230757195347, 0.0, 0.39832663551574843, 0.15033858031130232, 0.39370901552650506, 0.2791880535215935, 0.5624425234504322, 0.0, 0.21924544040742097, 0.0013101600376613414, 0.28442006448525226, 0.0007329887751062031, 0.015244881148666277, 0.0, 0.0015488784405916349, 0.12872303200906637, 0.3012417272786842, 0.0009642947166028688, 0.4233424090922028, 0.0, 0.5242999644393089, 0.2980053678174942, 0.1405386141665543, 0.2607509553000463, 0.4560551692516166, 0.02454314938777724, 0.4032244002383063, 0.43282963485534676, 0.38543578784004684, 0.45661307237481263, 0.004670268010047687, 0.15390786124884812, 0.0, 0.4275745876071248, 0.0, 0.40538629173989454, 0.4423304672092238, 0.6745234679237548, 0.044348037900721816, 0.3965067012229576, 0.24622472941480522, 0.05607326727550255, 0.2287069005010141, 0.004670268010047687, 0.501873096321369, 0.0, 0.44152368552472, 0.01033767367735455, 0.5155389977492297, 0.3553144129104062, 0.2779267001554299, 0.4260063654743079, 0.003057938283563648, 0.7160999045003817, 0.026355546734820347, 0.27017365176686925, 0.4788906772316454, 0.4239160142398002, 0.021773548862657158, 0.017372631960209817, 0.8538594437274869, 0.3869483924614638, 0.12783237330801137, 0.008583532425309721, 0.3993516070174706, 0.5271498802470692, 0.4560551692516166, 0.5101056434334685, 0.01206446861407128, 0.4138532153662632, 0.002652449642521827, 0.5200174076945469, 0.0, 0.0, 0.43024869077279276, 0.3550526379113277, 0.0015821730248192526, 0.23072755417956656, 0.003677688330765132, 0.1733234315973007, 0.08031143475771152, 0.09903175855548844, 0.1955019654022855, 0.6212996921007555, 0.6386563973456316, 0.6174486238224547, 0.09304383462238221, 0.09304383462238221, 0.2852911941444901, 0.6345074401522766, 0.26689970913573974, 0.669067131740209, 0.5752406242595747, 0.0, 0.5388925670278284, 0.03867130302707173, 0.05082332573460867, 0.017372631960209817, 0.057494581926314195]
display_histogram(groupvit_scores,'GroupVIT: Mask IOU')
display_histogram(groupvit_scores,'GroupVIT: Mask IOU',cum=True)
rp.ic(rp.median(groupvit_scores),rp.mean(groupvit_scores))

In [ ]:
clippy_scores=[0.45753623188405795, 0.41653922454728076, 0.4755779679676125, 0.171245339278172, 0.6187999694123534, 0.0, 0.28080790674663836, 0.07320851150045996, 0.4814878561476979, 0.22481701591355233, 0.17793566453026757, 0.1963647374105759, 0.125747211806845, 0.6266105305873998, 0.30207921305452473, 0.7573870412084271, 0.3446304367310101, 0.107212835408972, 0.2956283699709664, 0.5690553070762095, 0.3055664035415822, 0.2723764444370818, 0.36520416283783486, 0.41039273057755554, 0.32276735929309985, 0.787093943989842, 0.10446623393178052, 0.1493724187615762, 0.5210771244101501, 0.12309996866883327, 0.3397868398363693, 0.4447839741957389, 0.3422263352723427, 0.7719875360569969, 0.3768483081695806, 0.22161369265243544, 0.2424491432867544, 0.32018243144528646, 0.2516336392582881, 0.4862760686826771, 0.5680893475176012, 0.3824700105451205, 0.5623142280518388, 0.44295576424814714, 0.18058652050500726, 0.435527624804971, 0.38500435934932803, 0.2028134281149817, 0.5013516249477554, 0.44761780943990187, 0.06766807315578989, 0.3920583078117952, 0.6017213076201249, 0.3398328022525676, 5.639279461207128e-05, 0.40463673441172404, 0.32062953387219456, 0.38632430269775947, 0.6279047416147536, 0.3900421223955001, 0.3136442548766463, 0.25437221453818964, 0.24732678167523128, 0.34123557007585237, 0.1303289844320863, 0.47582066684456936, 0.24203800227870131, 0.34596792995363057, 0.26165755088377446, 0.3172196722662669, 0.3649685449486934, 0.43963715029799466, 0.26115554435129557, 0.47395708903291417, 0.2814988579541947, 0.1841062533298785, 0.5188652648698506, 0.46389120416942103, 0.42312077004180076, 0.8986277614416045, 0.0, 0.36600837719560153, 0.5203331184464998, 0.1630725837610025, 0.16648769613380696, 0.3328457407390701, 0.21934920454113985, 0.4922526217538519, 0.1748896282095424, 0.49791723370767904, 0.30165806312715593, 0.18804562661896376, 0.3630674824577954, 0.09046810699588477, 0.4688866718936815, 0.3724841765931517, 0.42492525504519096, 0.5080767502708988, 0.2727103814060336, 0.30310298091332916, 0.09225471882231578, 0.24182689055995868, 0.42506997790171525, 0.8739890502323872, 0.2776552679082079, 0.35297384972377405, 0.04270490946204589, 0.37346440872560277, 0.6154426641551812, 0.6433663725087507, 0.261082021133816, 0.06669359774758232, 0.3703382612979732, 0.33250390946953434, 0.027454806686956986, 0.26468423194663177, 0.0, 0.02147500499200426, 0.3750387493656585, 0.502741535092485, 0.39462659175184844, 0.3851221117292601, 0.3426998183469573, 0.0011410655262947972, 0.3348630632804488, 0.45094787454511664, 0.3831288816651936, 0.05845187951998729, 0.35885723596665126, 0.024278788842088533, 0.09046810699588477, 0.0, 0.0, 0.003023088377268339, 0.4638193870946142, 0.3595856351093551, 0.5865040977263711, 0.08407413985664529, 0.19019770009356657, 0.0, 0.3310789103254119, 0.25126890110597494, 0.38590861244527874, 0.3008104602571123, 0.3359753851055991, 0.3114464943719197, 0.22565427966289972, 0.354781038547552, 0.30748237616630764, 0.6769747600233035, 0.2688490205625679, 0.3779657098617749, 0.2699176778597355, 0.39992531006152093, 0.4232809902215278, 0.3315354006382767, 0.5140795806439578, 0.10595044873018904, 0.33362868678152063, 0.2970228349591281, 0.2935414741243275, 0.0, 0.4159163804635489, 0.38431589878650063, 0.47091098594287506, 0.23470917364086294, 0.2704430242554916, 0.2727804184860572, 0.3370215214980326, 0.4429580816504653, 0.30705669277623443, 0.41077325802699377, 0.4773039880441701, 0.3384478403637282, 0.11438322003479463, 0.2218485602559545, 0.49091200901972026, 0.23791650193056688, 0.3346096530296058, 0.06752264980146634, 0.232506151213707, 0.04213486944410639, 0.30832958011988615, 0.2568643531158194, 0.2922942319194476, 0.1581082659448021, 0.4537510711275474, 0.36259736951218086, 0.3642007487796787, 0.4310796591488709, 0.2747895765964432, 0.27371416665564274, 0.0010371235992571108, 0.3319608396016977, 0.3767338845482142, 0.04202374076126036, 0.26442647781725465, 0.37188227393938766, 0.16938416053343397, 0.634823685756467, 0.3955380939521668, 0.41702404007839405, 0.4131472010125868, 0.2729591928915484, 0.48012208741560486, 0.0, 0.5221749893881041, 0.3140748474639425, 0.3158055599629224, 0.12218086242471468, 0.39303127327451837, 0.2626780183112688, 0.28915471802542353, 0.3380851245821026, 0.4884397010502967, 0.07156496499231864, 0.4447912970286676, 0.0, 0.597492910246026, 0.49337057302752485, 0.15064911323404362, 0.3322053396424196, 0.3103695548770988, 0.3774105193026613, 0.4059586312107958, 0.002307297463497436, 0.07538126361655774, 0.3729108797798603, 0.15274249295339914, 0.410288550842127, 0.49380805791048327, 0.3762505420672599, 0.31275917487318206, 0.34391350308155577, 0.5114677333960821, 0.19741218913787992, 0.4431046685087556, 0.46516429288882366, 0.5501723651187908, 0.4507941266561956, 0.13295597414345633, 0.44726257695234356, 0.1609062225509685, 0.0, 0.11481928926725775, 0.4731661605450613, 0.5294585932145898, 0.37598582609477926, 0.3629515041225159, 0.44662106648980926, 0.06328675787441021, 0.40311244627971704, 0.0, 0.47249668586272225, 0.23597310475215716, 0.7792341206247244, 0.5585712405548312, 0.6275113503904609, 0.6866585603928147, 0.37131073958374616, 0.42192428037214597, 0.3282472674552877, 0.17390699126267925, 0.6184371668342297, 0.32657412427165006, 0.3443928748104682, 0.24247427794651563, 0.7014066827134612, 0.019384012370096012, 0.5483258173738453, 0.218591977181505, 0.07896564015356854, 0.3164914810458012, 0.43687970963934075, 0.06809398726206325, 0.33063014186694306, 0.408334116762245, 0.0, 0.3551841759246908, 0.418112469565526, 0.4071097070461984, 0.001354518203262388, 0.1440217015073568, 0.09934116909993101, 0.22475990910772461, 0.5340130196934053, 0.3271910336397119, 0.3620583382728183, 0.04390584538392156, 0.03657604360542409, 0.5787911552646088, 0.329792181182671, 0.4100247744157603, 0.03641844278385158, 0.477746220508551, 0.3351349382339415, 0.1926726211521408, 0.029534225407160237, 0.40934925945491185, 0.18085546226825946, 0.08180900235606195, 0.0, 0.1313619518221124, 0.7063777863182168, 0.22790367508732637, 0.3984381835878479, 0.2571820559789532, 0.0, 0.15417391806402125, 0.0, 0.18592786225049884, 0.44249930092572154, 0.3290294069476526, 0.4125801785194663, 0.032970601471947567, 0.0019770674325003007, 0.1053579996510512, 0.06768525174946083, 0.5327838908237843, 0.4187566585056568, 0.008680757869191171, 0.3722664880393304, 0.23960347377241203, 0.27344752117232846, 0.5930698257631052, 0.4351516101195534, 0.35110585471426387, 0.41769450833006116, 0.4077178818920062, 0.09941259061864237, 0.43835529089189373, 0.39037005607122693, 0.19725915199939176, 0.7655600393685387, 0.4045219785479526, 0.5194415959833303, 0.21904602127949271, 0.3093230773807655, 0.5822317107315708, 0.4444623628169402, 0.3618211748869951, 0.5489898400093425, 0.028169034305034132, 0.4023722589750742, 0.4698192944199334, 0.4277973817076226, 0.3719026655441818, 0.18703648961941255, 0.44066200071802053, 0.6078259802271855, 0.2023872274986943, 0.4285303739428433, 0.30571434107776074, 0.69435426131556, 0.25402408354742434, 0.4181809732703818, 0.10704409742009723, 0.3036438883230677, 0.5275752016126224, 0.44805682262478824, 0.336193649726619, 0.05497376700733852, 0.0, 0.0, 0.4123588289421795, 0.32390871972496255, 0.3628281417612889, 0.22567838642466967, 0.09691286467389805, 0.0, 0.0, 0.09716846093703013, 0.13685678145750532, 0.23059134698230316, 0.16494393484168665, 0.34359782961554514, 0.4652450196873416, 0.5020716146695687, 0.3769122511361905, 0.23978409485921748, 0.4068820409415684, 0.4158725592082215, 0.00017858509579814783, 0.2187695437393278, 0.29996433802833505, 0.48824343015214383, 0.20033890646181654, 0.0, 0.08311531208861585, 0.45099863996783823, 0.30452821413749864, 0.09792307512926457, 3.331493080012945e-05, 0.033623950563844804, 0.2495456582861334, 0.28281946462432583, 0.42042718777717647, 0.0, 0.31472593845150665, 0.4016723387788021, 0.2861014610232953, 0.12127170247500565, 0.03931333276261942, 0.6210720382383066, 0.011353575846932433, 0.33295046661880834, 0.16454925041933294, 0.3397714879607249, 0.08358373792500054, 0.15078837496792855, 0.524563897332721, 0.5746129849483027, 0.3430530008856189, 0.8577072550065034, 0.3186987787466696, 0.6319785598118639, 0.4444147807395846, 0.21262118281560174, 0.4252631149853518, 0.2197959170549291, 0.06846772445624943, 0.41144786601836847, 0.25086478021905717, 0.07004179782836803, 0.3267680842636466, 0.0, 0.40194385136310984, 0.4639680494925959, 0.7917828026115294, 0.4685208919155014, 0.3463206959411127, 0.42093792703009797, 0.3322056633070331, 0.07400439682411217, 0.4856618647741363, 0.3171761493116767, 0.6366701844952763, 0.1541620941929584, 0.2613876808392972, 0.22833989297572793, 0.7689914639939927, 0.0, 0.08769649428239988, 0.23535770764561126, 0.23407812553570384, 0.2908601024102544, 0.46010137985072785, 0.07737496095241402, 0.4692217011352861, 0.3965367500683087, 0.653468520073287, 0.0, 0.31680590356925364, 0.3909850975718862, 0.5142989238954182, 0.01085069008782936, 0.3146826800477111, 0.0, 0.3076655286549048, 0.4616543099662867, 0.41012772516062546, 0.4107344800904708, 0.2668776174675962, 0.010175956244827462, 0.38818835887935144, 0.26862745534125687, 0.0, 0.31387259542797624, 0.32739119851164084, 0.39096626831254133, 0.2574334825132334, 0.5476966667112632, 0.03254214432748934, 0.5112203634702698, 0.20207236225437564, 0.43182809556855356, 0.5608101952335782, 0.2377645068333346, 0.4562858834097052, 0.41335529778645, 0.09694387288523092, 0.3319402812408975, 0.16410169068935668, 0.3145070763541919, 0.2750908327088585, 0.5295928678266607, 0.3459694288379902, 0.3105546910522496, 0.376918683732728, 0.40409663116476713, 0.40061466951988467, 0.24887646355740778, 0.0, 0.1239833781854434, 0.8018527367609278, 0.0, 0.4494387955841457, 0.4391751631823123, 0.5437161578492721, 0.33880612847505337, 0.0014814978851562326, 0.4433635749247373, 0.34178790781442586, 0.40088063287696074, 0.6381244052048467, 0.3794012885938206, 0.5200066710562794, 0.13597186138534714, 0.31410532589998674, 0.09730335524040125, 0.3751021240264687, 0.3325990740174183, 0.14930828090262663, 0.2966174545454546, 0.4435221930282783, 0.31627578300379267, 0.27394559526468115, 0.10928666690271335, 0.45841248970077825, 0.28807249056420164, 0.27897968264910067, 0.4470930149065112, 0.291049983227105, 0.2808124741693158, 0.34520579104688004, 0.028942570029775693, 0.3679497064231066, 0.3135046812774522, 0.38402469878276174, 0.45185486532212865, 0.17302718753692933, 0.42476687544761554, 0.24548427594569577, 0.24558573973044612, 0.0, 0.5304696789536266, 0.5449759658896643, 0.4066740612092996, 0.03692267110038209, 0.2258595749934197, 0.4259580266337023, 0.08134632930365929, 0.6518039652335765, 0.40934543766888887, 0.1488992968321919, 0.27012990843819507, 0.2990080028381762, 0.5428309392791634, 0.0, 0.636436739472346, 0.11711640900589503, 0.20666142500813361, 0.47807759859492255, 0.5925615212527964, 0.01722019016190775, 0.1539000810138218, 0.44924060413878136, 0.3232180322371466, 0.30212326526952965, 0.14701701455993935, 0.5059907017932256, 0.22665036499147395, 0.2617669553987069, 0.09341573856293747, 0.0, 0.1892959039881021, 0.5360214973143352, 0.36562163138600856, 0.08465064605494613, 0.011407836120248637, 0.2283251765185695, 0.0, 0.07746589340965875, 0.024751170510831026, 0.12347292915818046, 0.5253856391052074, 0.34818884363974906, 0.5341101254695118, 0.4436297619396211, 0.511715798502597, 0.01791204384440583, 0.23107790721004615, 0.27163115420898054, 0.4907809524177255, 0.558052060353709, 0.3087410698204959, 0.1753669467937195, 0.36723524427252685, 0.6463429058335365, 0.5662133101616378, 0.19680630337556418, 0.2624779379308394, 0.14165931952295804, 0.5253818831035064, 0.09916042899643208, 0.06623377440920594, 0.44459332858905876, 0.65552524172887, 0.6217507024164008, 0.42036710465010885, 0.28910722574655734, 0.003046811286902626, 0.4111534558513828, 0.5075648779769303, 0.05813314651278553, 0.346514581059104, 0.38553232737459814, 0.0015436333699231613, 0.0006969587487540581, 0.19731264619745933, 0.1702494715074853, 0.15726707389210876, 0.437585707910147, 0.3147070337485656, 0.694702204834318, 0.7088742053056323, 0.34792515922676465, 0.41401739058331644, 0.23656614773128706, 0.12552641736424566, 0.45937037535315484, 0.40709327398779765, 0.06805139563564974, 0.0763728020826439, 0.4237921749658182, 0.3373175887192387, 0.44170642269489374, 0.47723325737142247, 0.530230609036743, 0.07243778485110995, 0.446373659181541, 0.5661157378724765, 0.5334284188022842, 0.14039587095928877, 0.45851286939942804, 0.710394801441368, 0.46237171124863957, 0.7287944791411998, 0.15129543796435171, 0.692584650336675, 0.8177157496410068, 0.18146348543702742, 0.0, 0.18295585073826, 0.6998502631004632, 0.1678325027347553, 0.49342977485246325, 0.0, 0.3800132114648259, 0.037837940896091514, 0.09360254692026958, 0.4986758207752206, 0.5694620197272593, 0.16333115993133823, 0.5351309034118918, 0.29619010356896097, 0.5331099295408014, 0.06581545099009635, 0.453605060712, 0.3368146508684803, 0.2710160063170971, 0.23049632197514164, 0.09918227859122636, 0.05941616981737332, 0.27648635368740787, 0.06062693338321777, 0.002506500146865243, 0.0, 0.001529175050301811, 0.14390815681840854, 0.3503892035318273, 0.3096086316679052, 0.4048101813937289, 0.5123060082275464, 0.0, 0.003716605408963739, 0.0, 0.2212089922802762, 0.24390327989331057, 0.358756710904062, 0.21358579639544353, 0.35353509572048625, 0.23663560742310422, 0.0, 0.0028655040741506293, 0.4086541997439373, 0.21802867143642, 0.2529265650403497, 0.2637991799570894, 0.11510162407070611, 0.420289268385776, 0.6977043673012318, 0.1405982291242643, 0.2377462918664603, 0.37236664815908505, 0.4502625884660883, 0.012295437866406944, 0.7845263473159387, 0.002027336511836106, 0.0, 0.010534862901043076, 0.20113334221731488, 0.009652307933439566, 0.0016154601692140679, 0.2378014203548187, 0.3744995601000736, 0.11656674850122752, 0.4118957320315759, 0.3874532214018408, 0.21182984830761167, 0.676022189719466, 0.43777835507220647, 0.7475760994454899, 0.007805562814599754, 0.3598292115145348, 0.613113116722496, 0.2438137112945605, 0.383433351070469, 0.4601980182362329, 0.4824880947535512, 0.19217878906911626, 0.03014617537972719, 0.13467643899963413, 0.7252746055269811, 0.28784165315071786, 0.40831170256981314, 0.20469844159549871, 0.2022246253669087, 0.25879840484647737, 0.6483069180236141, 0.40231711737838033, 0.33781587572484145, 0.21746201288633152, 0.33338587676617587, 0.6165183266127628, 0.435804481692516, 0.6382617148780564, 0.520064103600975, 0.533688094928523, 0.3589143041806209, 0.3842873206254016, 0.3941963841161278, 0.40215085937568756, 0.6883183720302376, 0.33037268623319893, 0.22403041317904426, 0.3025811762347104, 0.37417286439479447, 0.2254652962235824, 0.2309412715504086, 0.2741380062995722, 0.0, 0.10617855509830819, 0.0, 0.47588204606578033, 0.11531329686153248, 0.4032090145385982, 0.02974394091320404, 0.046204921894820776, 0.3612331868424557, 0.1908456140227203, 0.2634159207206458, 0.22244860882756365, 0.5555070983035468, 0.45438379617538666, 0.16899277416264122, 0.34464463058387484, 0.0, 0.0, 0.2289400069892197, 0.17521161026746185, 0.4206784687793738, 0.36236132931962917, 0.0162514678745177, 0.315310215482613, 0.4541304330530118, 0.32403386197548106, 0.4555880705795124, 0.07506315165636486, 0.5322282801065724, 0.274980459372007, 0.39208980418053946, 0.4375318339418858, 0.10156748140276302, 0.21502342835937935, 0.6982951094700319, 0.2850913005035944, 0.5371693460708898, 0.595724860149327, 0.0378582232529999, 0.681647297723749, 0.05564041967782078, 0.2387538486909467, 0.2728635783798075, 0.18511723636234598, 0.3475607434378049, 0.565946043628111, 0.30362475872440703, 0.3515244993788297, 0.48693071195965604, 0.4627193841994622, 0.5047510047643129, 0.030349363875025244, 0.35483105835434997, 0.4101838862655162, 0.4400363711813311, 0.2812316864551752, 0.21876405515298822, 0.29936681070013105, 0.19932652374003818, 0.21312303455917098, 0.2897199313601663, 0.4055603710363921, 0.3747039569320628, 0.19711252625320638, 0.4300529800565415, 0.45460481618479653, 0.39868737246735864, 0.38825732715349814, 0.09766811533249217, 0.3148723494850752, 0.4373634388389097, 0.4935880787653007, 0.4611258867805515, 0.28316651067035525, 0.4137253138533759, 0.15606954756012437, 0.4471807079906247, 0.34916242016048316, 0.5049264234080788, 0.1590596678907761, 0.48562169471251515, 0.6345243058058733, 0.14010585523592606, 0.7053257977972855, 0.4065238690888812, 0.34435645670648735, 0.39616238399982673, 0.4594704565931351, 0.27879124142350314, 0.10617855509830819, 0.008186103414774567, 0.10818072702331961, 0.286446781160427, 0.05429920497459222, 0.024945604803940858, 0.42343515144640026, 0.2572012094626614, 0.36844446776618534, 0.5107972562887718, 0.31393579055592624, 0.01734367868553172, 0.33809716518280925, 0.38914167601391153, 0.33349729778073056, 0.37315643552624317, 0.303253986953735, 0.4209133852915855, 0.33217131651980725, 0.3238965798628509, 0.05812076563468703, 0.286096188500901, 0.25871978993556755, 0.09384410194013114, 0.472781306626869, 0.4180039474813353, 0.1294967612434021, 0.15606673849110445, 0.4243356043980041, 0.05232122639069555, 0.3644746186134395, 0.3034041652273715, 0.3814304794362824, 0.2779702701233763, 0.02746335963923337, 0.0016434471572660773, 0.20443733263004138, 0.7368542763626014, 0.3581788527753201, 0.452227136834759, 0.045031275286743645, 0.38516852433393217, 0.06276546267552756, 0.05656403600485513]
display_histogram(clippy_scores,'Clippy: Mask IOU')
display_histogram(clippy_scores,'Clippy: Mask IOU',cum=True)
rp.ic(rp.median(clippy_scores),rp.mean(clippy_scores))